## Open notebook in: 
| Colab                                 |  Gradient                                                                                                                                         |
|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Nicolepcx/Transformers-in-Action/blob/main/CH03/ch03_text_summarization_eval.ipynb)                                              | [![Gradient](https://assets.paperspace.io/img/gradient-badge.svg)](https://console.paperspace.com/github/Nicolepcx/Transformers-in-Action/blob/main/CH03/ch03_text_summarization_eval.ipynb)|             

# Download repo

In [1]:
# Clone repo, if it's not already cloned, to be sure all runs on smoothly
# on Colab, Kaggle or Paperspace
import os

if not os.path.isdir('Transformers-in-Action'):
    !git clone https://github.com/Nicolepcx/Transformers-in-Action.git
else:
    print('Repository already exists. Skipping clone.')


current_path = %pwd
if '/Transformers-in-Action' in current_path:
    new_path = current_path + '/utils'
else:
    new_path = current_path + '/Transformers-in-Action/utils'
%cd $new_path


Cloning into 'Transformers-in-Action'...
remote: Enumerating objects: 119, done.
remote: Counting objects: 100% (119/119), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 119 (delta 54), reused 75 (delta 30), pack-reused 0
Receiving objects: 100% (119/119), 573.37 KiB | 13.03 MiB/s, done.
Resolving deltas: 100% (54/54), done.
/content/Transformers-in-Action/utils


# About this notebook

In this notebook, we will explore and evaluate various text summarization models on the [BillSum dataset](https://huggingface.co/datasets/billsum). We will compare several state-of-the-art Transformer-based models, such as BART, T5, and Pegasus, as well as a TextRank-based baseline model. Our goal is to determine their performance and suitability for summarizing legislative text data.

The BillSum dataset is a collection of U.S. Congressional and California state bills with their corresponding summaries. It is an interesting use case for text summarization because legislative documents tend to be lengthy, complex, and written in a formal style. Moreover, they often contain domain-specific terminology and jargon, making it challenging to generate concise and accurate summaries.

In this notebook, we will:

1. Provide an overview of the different summarization models, including their strengths and limitations.
2. Implement and demonstrate the use of these models to generate summaries of the legislative text.
3. Evaluate the models on multiple samples from the BillSum dataset, considering both ROUGE and BLEU metrics for a comprehensive comparison.
4. Discuss the importance of evaluating models on a diverse set of samples to ensure a more reliable and generalizable performance assessment.
5. Provide insights into the BillSum dataset and why it is an interesting use case for text summarization.

By the end of this notebook, you will gain a better understanding of the various text summarization models, their performance on legislative text data, and the challenges associated with evaluating models on complex and domain-specific datasets like BillSum.

# Install requirements

In [2]:
from requirements import *

In [3]:
install_base_packages()
install_required_packages_ch03()

Installing base requirements...

✅ transformers==4.26.1 installation completed successfully!

✅ datasets==2.10.1 installation completed successfully!

Installing chapter 3 requirements...

✅ summa==1.2.0 installation completed successfully!

✅ evaluate==0.4.0 installation completed successfully!

✅ rouge_score==0.1.2 installation completed successfully!

✅ sentencepiece installation completed successfully!



# Imports

In [4]:
from setup import *
from utils import *
from textrank import *

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
useGPU()

Have fun with this chapter!🥳


In [6]:
import pandas as pd
import numpy as np
from functools import partial
import logging

import nltk
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from summa import summarizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline, GenerationConfig, set_seed
from transformers import BigBirdPegasusForConditionalGeneration, AutoTokenizer
from transformers import LEDForConditionalGeneration, LEDTokenizer
import torch
from datasets import load_dataset

from multiprocessing import Pool
import evaluate
from tqdm import tqdm

import matplotlib.pyplot as plt

from textwrap import TextWrapper

set_seed(42)

# Allocate enough RAM

Let us try to get a __GPU__ with at least __15GB RAM__ for our notebook.

In [7]:
# crash colab to get more RAM -> uncomment to use
# !kill -9 -1

We can execute the following command `!free -h`  to see if we have enough RAM and `!nvidia-smi` to get more info about our GPU type we got assigned.
If the allocated GPU is too small, the above cell can be used to run the command to crash the notebook hoping to get a better GPU after the crash, since the GPU is randomly allocated.

In [8]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:           51Gi       1.3Gi        45Gi       2.0Mi       4.4Gi        49Gi
Swap:            0B          0B          0B


In [9]:
!nvidia-smi

Wed Apr  5 06:01:35 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    12W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [10]:
if torch.cuda.is_available():
    gpu_device = torch.device('cuda')
    gpu_info = torch.cuda.get_device_properties(gpu_device)
    gpu_memory = gpu_info.total_memory / 1e9  # Convert bytes to gigabytes
    print(f"GPU: {gpu_info.name}, Total Memory: {gpu_memory:.2f} GB")
else:
    print("No GPU detected.")


GPU: Tesla T4, Total Memory: 15.84 GB


# Use case BillSum dataset

## Load the BillSum dataset 

In [11]:
# Load the dataset
dataset = load_dataset("billsum")
#test_dataset.set_format(type='pandas', columns=['article', 'abstract'])

Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

Dataset billsum downloaded and prepared to /root/.cache/huggingface/datasets/billsum/default/3.0.0/75cf1719d38d6553aa0e0714c393c74579b083ae6e164b2543684e3e92e0c4cc. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [12]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 18949
    })
    test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 3269
    })
    ca_test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 1237
    })
})

In [13]:
# Get the test split of the dataset
dataset_test = load_dataset("billsum", split="ca_test")
dataset_test.set_format(type="pandas", columns=['text', 'summary'])

In [14]:
# Convert to pandas DataFrame
df = dataset_test.to_pandas()

## Excerpt of article and ground truth of summary




In [15]:
# Access first row using iloc
sample = df.iloc[0]

excerpt = 1000

print(f"\033[1mExcerpt of {excerpt} characters, total length of article: \
{len(sample['text'])}:\033[0m\n")

print(sample["text"][:excerpt])
print(f"\033[1m\n\nSummary (length: {len(sample['summary'])}):\033[0m\n")
print(sample["summary"])


Excerpt of 1000 characters, total length of article: 8203:

The people of the State of California do enact as follows:


SECTION 1.
The Legislature finds and declares all of the following:
(a) (1) Since 1899 congressionally chartered veterans’ organizations
have provided a valuable service to our nation’s returning service
members. These organizations help preserve the memories and incidents
of the great hostilities fought by our nation, and preserve and
strengthen comradeship among members.
(2) These veterans’ organizations also own and manage various
properties including lodges, posts, and fraternal halls. These
properties act as a safe haven where veterans of all ages and their
families can gather together to find camaraderie and fellowship, share
stories, and seek support from people who understand their unique
experiences. This aids in the healing process for these returning
veterans, and ensures their health and happiness.
(b) As a result of congressional chartering of these vete

In [16]:
articles_np= np.array(dataset_test['text'])
articles_np.shape

(1237,)

In [17]:
articles_np_1 = articles_np[:1]
articles_np_1.shape

(1,)

In [18]:
print(articles_np_1)

['The people of the State of California do enact as
follows:\n\n\nSECTION 1.\nThe Legislature finds and declares all of
the following:\n(a) (1) Since 1899 congressionally chartered veterans’
organizations have provided a valuable service to our nation’s
returning service members. These organizations help preserve the
memories and incidents of the great hostilities fought by our nation,
and preserve and strengthen comradeship among members.\n(2) These
veterans’ organizations also own and manage various properties
including lodges, posts, and fraternal halls. These properties act as
a safe haven where veterans of all ages and their families can gather
together to find camaraderie and fellowship, share stories, and seek
support from people who understand their unique experiences. This aids
in the healing process for these returning veterans, and ensures their
health and happiness.\n(b) As a result of congressional chartering of
these veterans’ organizations, the United States Internal Rev

## Applying TextRank algorithm to BillSum dataset

This code snippet is used to generate summaries for a collection of texts (in this case, legislative documents) using the TextRank algorithm. The TextRank summarization is implemented with the summa library, which is available on [PyPI](https://pypi.org/project/summa/). The implementation is based on the paper Mihalcea, R., Tarau, P.: [“TextRank: Bringing order into texts”](https://aclanthology.org/W04-3252/)

A `summarize_text` function is defined, which takes in two arguments - text and words. text is the input text to be summarized, and words is an optional parameter to specify the target number of words in the generated summary (default is 250 words). The `summarizer.summarize()` function from the summa library is used to generate the summary.

The code then employs the multiprocessing Pool class to parallelize the summarization process. This speeds up the computation by taking advantage of multiple CPU cores available on the system. The `Pool()` context manager creates a pool of worker processes, and the `pool.map()` function is used to apply the `summarize_text` function to each element of the `articles_np_1` array (which contains the input articles).

The `pool.map()` function distributes the articles across the available worker processes, and each process applies the summarize_text function independently. Once all the summaries are generated, they are collected and stored in the `summarized_articles` list. This parallelization can significantly improve the performance of the TextRank summarization, especially when working with large datasets.

In [19]:
# Define the function to summarize the text
def summarize_text(text, words=250):
    summary = summarizer.summarize(text, words=words)
    return summary

# Parallelize the TextRank summarization
with Pool() as pool:
    summarized_articles = pool.map(summarize_text, articles_np_1)


### Get summarized article

In [20]:
print(summarized_articles)

['(c) Section 501(c)(19) of the Internal Revenue Code and related
federal regulations provide for the exemption for posts or
organizations of war veterans, or an auxiliary unit or society of, or
a trust or foundation for, any such post or organization that, among
other attributes, carries on programs to perpetuate the memory of
deceased veterans and members of the Armed Forces and to comfort their
survivors, conducts programs for religious, charitable, scientific,
literary, or educational purposes, sponsors or participates in
activities of a patriotic nature, and provides social and recreational
activities for their members.\n(a) All buildings, and so much of the
real property on which the buildings are situated as may be required
for the convenient use and occupation of the buildings, used
exclusively for charitable purposes, owned by a veterans’ organization
that has been chartered by the Congress of the United States,
organized and operated for charitable purposes, and exempt from
f

In [21]:
references = np.array(dataset_test['summary'])

In [22]:
references_np = np.array(references)
print(references_np[:1])

['Existing property tax law establishes a veterans’ organization
exemption under which property is exempt from taxation if, among other
things, that property is used exclusively for charitable purposes and
is owned by a veterans’ organization.\nThis bill would provide that
the veterans’ organization exemption shall not be denied to a property
on the basis that the property is used for fraternal, lodge, or social
club purposes, and would make specific findings and declarations in
that regard. The bill would also provide that the exemption shall not
apply to any portion of a property that consists of a bar where
alcoholic beverages are served.\nSection 2229 of the Revenue and
Taxation Code requires the Legislature to reimburse local agencies
annually for certain property tax revenues lost as a result of any
exemption or classification of property for purposes of ad valorem
property taxation.\nThis bill would provide that, notwithstanding
Section 2229 of the Revenue and Taxation Code, no 

In [23]:
reference = references[0]

In [24]:
print(summarized_articles[0])

(c) Section 501(c)(19) of the Internal Revenue Code and related
federal regulations provide for the exemption for posts or
organizations of war veterans, or an auxiliary unit or society of, or
a trust or foundation for, any such post or organization that, among
other attributes, carries on programs to perpetuate the memory of
deceased veterans and members of the Armed Forces and to comfort their
survivors, conducts programs for religious, charitable, scientific,
literary, or educational purposes, sponsors or participates in
activities of a patriotic nature, and provides social and recreational
activities for their members.
(a) All buildings, and so much of the real property on which the
buildings are situated as may be required for the convenient use and
occupation of the buildings, used exclusively for charitable purposes,
owned by a veterans’ organization that has been chartered by the
Congress of the United States, organized and operated for charitable
purposes, and exempt from fede

In [25]:
# Create a dict to hold our summaries
summaries = {}
summaries['TextRank (Baseline)'] = summarized_articles

In [26]:
print(summaries['TextRank (Baseline)'][0])

(c) Section 501(c)(19) of the Internal Revenue Code and related
federal regulations provide for the exemption for posts or
organizations of war veterans, or an auxiliary unit or society of, or
a trust or foundation for, any such post or organization that, among
other attributes, carries on programs to perpetuate the memory of
deceased veterans and members of the Armed Forces and to comfort their
survivors, conducts programs for religious, charitable, scientific,
literary, or educational purposes, sponsors or participates in
activities of a patriotic nature, and provides social and recreational
activities for their members.
(a) All buildings, and so much of the real property on which the
buildings are situated as may be required for the convenient use and
occupation of the buildings, used exclusively for charitable purposes,
owned by a veterans’ organization that has been chartered by the
Congress of the United States, organized and operated for charitable
purposes, and exempt from fede

# Appyling various Transformer models to the BillSum dataset

Here we introduce a function called `generate_summary` that allows you to generate a summary of a given input text using various pre-trained transformer models. The function supports the following models: 

- BART [Modelcard](https://huggingface.co/facebook/bart-large-cnn), [Paper](https://arxiv.org/abs/1910.13461)
- T5 [Modelcard](https://huggingface.co/sysresearch101/t5-large-finetuned-xsum-cnn), [Paper](https://arxiv.org/abs/1910.10683)
- ProphetNet [Modelcard](), [Paper](https://arxiv.org/abs/2001.04063)
- Pegasus [Modelcard](https://huggingface.co/google/pegasus-cnn_dailymail), [Paper](https://arxiv.org/abs/1912.08777)

__Function: generate_summary__ <br>
The `generate_summary` function takes two arguments: `model_name` and `sample_text`. The model_name argument is a string that specifies the transformer model to use for summarization, while the `sample_text` argument is the input text to be summarized.

__Usage__ 

To use the generate_summary function, simply call it with the desired model name and the input text:

`summary = generate_summary("BART", "This is an example text to summarize using BART.")
print(summary)
`

__Models and Tokenizers__

The function sets up the tokenizer and model using the `AutoTokenizer` and `AutoModelForSeq2SeqLM` classes from the Hugging Face Transformers library. The model and tokenizer are specified in the `model_dict` dictionary:

`model_dict = {
    "BART": "facebook/bart-large-cnn",
    "T5": "sysresearch101/t5-large-finetuned-xsum-cnn",
    "ProphetNet": "microsoft/prophetnet-large-uncased-cnndm"
}
`

__Handling Text Chunks__

Some transformer models have a __maximum input length constraint__. To handle this, the input text is broken down into smaller chunks before being passed to the pipeline. The chunks are processed one by one, and the generated summaries are concatenated to form the final summary.


__Error Handling__

If the specified model_name is not supported, the function raises a ValueError.

__Example__

Here's an example of how to use the generate_summary function:

`summary = generate_summary("BART", "This is an example text to summarize using BART.")
print(summary)
`

In [27]:
def generate_summary(model_name, sample_text):
    """
    Generate a summary of the input text using a specified model.

    Args:
        model_name (str): The name of the model to use for summarization. Supported values are "BART", "T5",
            "ProphetNet", "Pegasus", and "GPT-2".
        sample_text (str): The input text to summarize.

    Returns:
        str: The summary generated by the specified model.

    Raises:
        ValueError: If the specified model is not supported.

    Notes:
        - If `model_name` is "Pegasus", the `google/pegasus-cnn_dailymail` model will be used for summarization.
          This model doesn't require a tokenizer or a maximum length.
        - If `model_name` is "GPT-2", the `gpt2-xl` model will be used for text generation. The summary will be
          generated by extracting the first paragraph from the generated text that follows the "TL;DR:" token.

        - For other models, the `generate_summary` function uses the `pipeline` method from the transformers library to
          generate the summary. Since some of these models have a maximum input length constraint, the input text needs to
          be broken down into smaller chunks before being passed to the `pipeline`. The size of the chunks is determined
          by the `max_length` parameter of the tokenizer used for the specified model. The chunks are processed one by one,
          and the generated summaries are concatenated to form the final summary.

    Example:
        >>> generate_summary("BART", "This is an example text to summarize using BART.")
        'BART is used to summarize the input text.'
    """

    # Set up tokenizer and model
    model_dict = {
        "BART": "facebook/bart-large-cnn",
        "T5": "sysresearch101/t5-large-finetuned-xsum-cnn",
        "ProphetNet": "microsoft/prophetnet-large-uncased-cnndm"
    }
    
    if model_name == "Pegasus":
        # Generate summary using the specified model
        summarization_pipeline = pipeline("summarization", model="google/pegasus-cnn_dailymail")
        chunks = [sample_text[i:i+2048] for i in range(0, len(sample_text), 2048)]
        summaries = []
        for chunk in chunks:
            summarization_output = summarization_pipeline(chunk, max_length=512)
            summaries.append(summarization_output[0]["summary_text"].replace(" .<n>", ".\n"))
        summary = " ".join(summaries)
    elif model_name in model_dict.keys():
        tokenizer = AutoTokenizer.from_pretrained(model_dict[model_name], max_length=1024, truncation=True)
        model = AutoModelForSeq2SeqLM.from_pretrained(model_dict[model_name])
        gen_config = GenerationConfig(max_length=512)
        summarization_pipeline = pipeline("summarization", tokenizer=tokenizer, model=model, config=gen_config)
        # Generate summary using the specified model
        chunks = [sample_text[i:i+1024] for i in range(0, len(sample_text), 1024)]
        summaries = []
        for chunk in chunks:
            summarization_output = summarization_pipeline(chunk)
            summaries.append(summarization_output[0]["summary_text"])
        summary = " ".join(summaries)
    else:
        raise ValueError(f"Model {model_name} is not supported.")
    
    return summary


In [28]:
# Hide transformers output 
logging.getLogger("transformers").setLevel(logging.ERROR)

## Generate summary from  <font color='red'>"Pegasus"</font> model

In [29]:
sample_text = sample[0]
summaries["Pegasus"] = generate_summary("Pegasus", sample_text)

## Generate summary from  <font color='red'>"BART"</font> model

In [30]:
summaries["BART"] = generate_summary("BART", sample_text)

## Generate summary from  <font color='red'>"T5"</font> model

In [31]:
summaries["T5"] = generate_summary("T5", sample_text)

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1186: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


## Generate summary from  <font color='red'>"ProphetNet"</font> model

In [32]:
summaries["ProphetNet"] = generate_summary("ProphetNet", sample_text)

## Generate summary from  <font color='red'>"BigBirdPegasus"</font> model

BigBird, is a sparse-attention based transformer which extends Transformer based models, such as BERT to much longer sequences. Moreover, BigBird comes along with a theoretical understanding of the capabilities of a complete transformer that the sparse model can handle. BigBird was introduced in this [paper](https://arxiv.org/abs/2007.14062), [Modelcard](https://huggingface.co/google/bigbird-pegasus-large-arxiv). 
<br><br>

__Model description__

BigBird relies on block sparse attention instead of normal attention (i.e. BERT's attention) and can handle sequences up to a length of 4096 at a much lower compute cost compared to BERT. It has achieved SOTA on various tasks involving very long sequences such as long documents summarization, question-answering with long contexts.

The models checkpoint is obtained after fine-tuning BigBirdPegasusForConditionalGeneration for summarization on arxiv dataset from scientific papers.

In [33]:

tokenizer = AutoTokenizer.from_pretrained("google/bigbird-pegasus-large-arxiv")
model = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-arxiv", attention_type="original_full", block_size=16, num_random_blocks=2)

text = sample_text
inputs = tokenizer.encode_plus(text, return_tensors='pt', max_length=4096, truncation=True)
prediction = model.generate(**inputs, max_length=512)  # Set max_length here
prediction = tokenizer.batch_decode(prediction)


In [34]:
summaries['bigbird-pegasus'] = prediction

In [35]:
summaries['bigbird-pegasus'] = summaries['bigbird-pegasus'][0]

In [36]:
print_summaries(summaries, reference)

Ground truth
Existing property tax law establishes a veterans’ organization
exemption under which property is exempt from taxation if, among other
things, that property is used exclusively for charitable purposes and
is owned by a veterans’ organization.
This bill would provide that the veterans’ organization exemption
shall not be denied to a property on the basis that the property is
used for fraternal, lodge, or social club purposes, and would make
specific findings and declarations in that regard. The bill would also
provide that the exemption shall not apply to any portion of a
property that consists of a bar where alcoholic beverages are served.
Section 2229 of the Revenue and Taxation Code requires the Legislature
to reimburse local agencies annually for certain property tax revenues
lost as a result of any exemption or classification of property for
purposes of ad valorem property taxation.
This bill would provide that, notwithstanding Section 2229 of the
Revenue and Taxation C

### Evaluating Summarization Models using ROUGE and BLEU Metrics

In this section, we demonstrate how to evaluate the summarization models using both the ROUGE and BLEU metrics. We use the Hugging Face's implementation of these metrics.

ROUGE, or Recall-Oriented Understudy for Gisting Evaluation, is a set of metrics and a software package used for evaluating automatic summarization and machine translation software in natural language processing. The metrics compare an automatically produced summary or translation against a reference or a set of references (human-produced) summary or translation. Note that ROUGE is case insensitive, meaning that upper case letters are treated the same way as lower case letters.
[Link](https://huggingface.co/spaces/evaluate-metric/rouge)

BLEU, or Bilingual Evaluation Understudy, is an evaluation metric used primarily for machine translation. It measures the similarity between a candidate translation and a set of reference translations, considering both the n-gram precision and a brevity penalty factor.
[Link](https://huggingface.co/spaces/evaluate-metric/google_bleu)

[Info about Huggingface evaluate](https://huggingface.co/docs/evaluate/v0.4.0/en/package_reference/loading_methods#evaluate.load)
<br> <br>

__Note:__ It can be beneficial to use both ROUGE and BLEU metrics for evaluation, as they focus on different aspects of the generated summaries. ROUGE is recall-oriented and mainly focuses on the coverage of important content, while BLEU is precision-oriented and evaluates the fluency and correctness of the generated text.

It is important to have an understanding of what constitutes a "good" score when evaluating summarization models. Here are some guidelines for interpreting the ROUGE and BLEU scores:

__ROUGE Scores__

ROUGE scores usually range from 0 to 1, with 1 being a perfect match between the generated summary and the reference summary. In practice, scores closer to 1 are rare, especially for abstractive summarization tasks.

For a simple baseline model like TextRank or LSA, a "good" ROUGE score can be around 0.3 to 0.4. For more advanced Transformer-based models like BART, T5, or Pegasus, a "good" ROUGE score can range from 0.4 to 0.6, depending on the task and dataset. State-of-the-art models can achieve even higher scores, but it's important to keep in mind that ROUGE scores are just one aspect of evaluating the quality of generated summaries.

__BLEU Scores__

BLEU scores also range from 0 to 1, with 1 indicating a perfect match between the generated text and the reference text. However, BLEU scores tend to be lower compared to ROUGE scores, as they are more sensitive to the differences in word order and phrasing.

For a simple baseline model, a "good" BLEU score could be around 0.1 to 0.2. For Transformer-based models, a "good" BLEU score might range from 0.2 to 0.4, again depending on the task and dataset. State-of-the-art models can achieve higher scores, but just like with ROUGE, it's essential to consider other factors when evaluating the quality of generated summaries.

Keep in mind that these are rough guidelines, and the definition of a "good" score can vary depending on the specific domain, dataset, and evaluation criteria. It is always a good idea to compare your model's scores with scores from existing state-of-the-art models on the same dataset to get a better understanding of its performance.

__Defining the SummarizationMetrics Class__

The SummarizationMetrics class is defined with methods for computing the ROUGE and BLEU metrics, as well as a method to compute both metrics together:

`class SummarizationMetrics:
    ...
`

This class is located in the `utils.py` file, feel free to look it up to get a better understanding how the mectrics are computed.

__Computing Metrics__

To compute the metrics, we first instantiate the SummarizationMetrics class:
`evaluator = SummarizationMetrics()
`

Then, we call the compute_sum_metric method on the evaluator object, passing the generated summaries and the reference summary:
`metrics_df = evaluator.compute_sum_metric(summaries, reference)
`

The compute_sum_metric method computes both the ROUGE and BLEU metrics by calling the `compute_rouge_metrics` and `compute_google_bleu_metrics` methods, respectively. The resulting metrics are combined into a single DataFrame and returned.

__Viewing the Metrics__

We can display the resulting metrics DataFrame to see the evaluation scores for each model:
`metrics_df
`

This DataFrame shows the ROUGE and BLEU scores for each summarization model, allowing for an easy comparison of their performance.


__Note:__ It's crucial to understand that evaluating a summarization model based on just one text example from a testing dataset is not enough to determine its overall performance. The reason behind this is that the complexity of the text samples can vary significantly within a dataset. Some text samples might be easier for a model to summarize, while others could be more challenging, containing complex sentence structures, domain-specific jargon, or long and convoluted narratives.

To get a more comprehensive sense of a model's performance, it is essential to evaluate it on a larger number of samples from the testing dataset. By doing so, we can ensure that we're assessing the model's ability to handle a diverse set of inputs, including those that differ in complexity, topic, and structure. This approach provides a more reliable and generalizable evaluation of the model's performance, helping us to better understand its strengths and weaknesses.

In the next section of this notebook, we will expand our evaluation to include multiple text samples from the testing dataset. This will give us a better understanding of how well the model performs across a variety of inputs and allow us to make more informed decisions about its applicability to real-world use cases.

In [37]:
evaluator = SummarizationMetrics()

metrics_df = evaluator.compute_sum_metric(summaries, reference)

metrics_df


100%|██████████| 6/6 [00:00<00:00, 191.18it/s]


,rouge1,rouge2,rougeL,rougeLsum,google_bleu
TextRank (Baseline),0.414815,0.094293,0.177778,0.340741,0.128283
Pegasus,0.493151,0.224771,0.287671,0.438356,0.218349
BART,0.374456,0.197962,0.220610,0.328012,0.143989
T5,0.460829,0.203704,0.230415,0.368664,0.184557
ProphetNet,0.401198,0.156627,0.179641,0.317365,0.139651
bigbird-pegasus,0.118280,0.000000,0.107527,0.118280,0.030905


## Evaluate more than one example

The `generate_avg_summary` is almost identical to `generate_summary`. However the main differences are:

1. The choice of models: While both functions support "BART", "T5", "ProphetNet", and "Pegasus", the generate_avg_summary function also supports the "BigBirdPegasus" model, which is designed for handling longer input sequences.

2. Handling long input text: The generate_avg_summary function takes advantage of the BigBirdPegasus model's ability to handle longer input sequences (up to 4096 tokens). This can be particularly useful for summarizing long documents or articles.

In [38]:
def generate_avg_summary(model_name, sample_text):
    """
    This function generates an average summary for the given text using the 
    specified transformer model. Different transformer models such as Pegasus, 
    T5, and others are supported.
    Each model has its own tokenizer and configuration settings. For example, 
    Pegasus and T5 use a maximum length of 1024, while BigBirdPegasus has a 
    maximum length of 4096. The input text is chunked according to the model's 
    maximum length and processed accordingly. The final summary is obtained by 
    concatenating the summaries generated for each chunk.


    Args:
        model_name (str): The name of the model to use for summarization. Supported values are "BART", "T5",
            "ProphetNet", "Pegasus", and "BigBirdPegasus".
        sample_text (str): The input text to summarize.

    Returns:
        str: The summary generated by the specified model.

    Raises:
        ValueError: If the specified model is not supported.

    Example:
        >>> generate_summary("BART", "This is an example text to summarize using BART.")
        'BART is used to summarize the input text.'
    """

    # Set up tokenizer and model
    model_dict = {
        "BART": ("facebook/bart-large-cnn", None),
        "T5": ("t5-large", None),
        "ProphetNet": ("microsoft/prophetnet-large-uncased-cnndm", None),
        "Pegasus": ("google/pegasus-cnn_dailymail", 1024),
        "BigBirdPegasus": ("google/bigbird-pegasus-large-arxiv", 4096),
    }
    
    if model_name in model_dict:
        if model_name == "Pegasus":
            tokenizer = AutoTokenizer.from_pretrained(model_dict[model_name][0], max_length=1024, truncation=True)
            model = AutoModelForSeq2SeqLM.from_pretrained(model_dict[model_name][0])
            summarization_pipeline = pipeline("summarization", tokenizer=tokenizer, model=model, max_length=model_dict[model_name][1])
            chunks = [sample_text[i:i+model_dict[model_name][1]] for i in range(0, len(sample_text), model_dict[model_name][1])]
            summaries = []
            for chunk in tqdm(chunks):
                summarization_output = summarization_pipeline(chunk)
                summaries.append(summarization_output[0]["summary_text"])
            summary = " ".join(summaries)
        elif model_name == "BigBirdPegasus":
            tokenizer = AutoTokenizer.from_pretrained(model_dict[model_name][0], max_length=4096, truncation=True)
            model = AutoModelForSeq2SeqLM.from_pretrained(model_dict[model_name][0])
            inputs = tokenizer(sample_text, return_tensors='pt', max_length=4096, truncation=True)
            outputs = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"])
            summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
        else:
            tokenizer = AutoTokenizer.from_pretrained(model_dict[model_name][0], model_max_length=1024, truncation=True)
            model = AutoModelForSeq2SeqLM.from_pretrained(model_dict[model_name][0])
            if model_name == "T5":
                gen_config = GenerationConfig(max_length=1024)
                summarization_pipeline = pipeline("summarization", tokenizer=tokenizer, model=model, config=gen_config)
            else:
                summarization_pipeline = pipeline("summarization", tokenizer=tokenizer, model=model, max_length=512)
            chunks = [sample_text[i:i+1024] for i in range(0, len(sample_text), 1024)]
            summaries = []
            for chunk in tqdm(chunks):
                summarization_output = summarization_pipeline(chunk)
                summaries.append(summarization_output[0]["summary_text"])
            summary = " ".join(summaries)


    else:
        raise ValueError(f"Model {model_name} is not supported.")
    
    return summary


## Summarizing a Small Dataset using Transformer variants and `generate_avg_summary`

In this section, we demonstrate how to use the `generate_avg_summary` function to generate summaries for a small dataset using the various models. The dataset in this example is assumed to be in the `dataset_test`
 variable, which is an instance of Hugging Face's Dataset class.

__Selecting a Small Range of Samples__

First, we select a small range of samples from the dataset. In this example, we select the first 10 samples:
`# Select a small range of samples
dataset_small = dataset_test.select(range(10))
`

__Converting the Dataset to a Pandas DataFrame__
Next, we convert the small dataset to a Pandas DataFrame, which provides an easier way to apply the generate_avg_summary function on each article:
`# Convert dataset to pandas dataframe
df = dataset_small.to_pandas()
`

__Generating Summaries using BART, T5, Pegasus and our baseline, TextRank__

Now that we have a DataFrame containing the articles, we can apply the __generate_avg_summary__ function to each article to generate BART-based summaries. For each article, a progress bar will appear, indicating the progress of the summarization process. Since we have selected 10 articles, 10 progress bars will be displayed.
`# Generate summaries for each article with BART
df["BART_avg"] = df["text"].apply(lambda x: generate_avg_summary("BART", x))
`

After the summarization process is complete, the resulting DataFrame df will contain a new column named "Modl_avg" with the Model-generated summaries for each article. You can adjust the `.select(range(10)` with different ranges to evaluate less or more articles. 

In [39]:
# Select a small range of samples
# See: https://huggingface.co/docs/datasets/v1.1.1/processing.html
dataset_small = dataset_test.select(range(10))

# Convert dataset to pandas dataframe
df = dataset_small.to_pandas()

# Note: every bar appearing is == to one data iteration, so if you have
# dataset_test.select(range(10)) 10 bars will apprear
# Generate summaries for each article with BART
df["BART_avg"] = df["text"].apply(lambda x: generate_avg_summary("BART", x))


100%|██████████| 10/10 [00:47<00:00,  4.78s/it]


In [40]:
# Generate summaries for each article with T5
df["T5_avg"] = df["text"].apply(lambda x: generate_avg_summary("T5", x))

  0%|          | 0/9 [00:00<?, ?it/s]/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1186: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
100%|██████████| 10/10 [01:34<00:00,  9.41s/it]


In [41]:
# Generate summaries for each article with Pegasus
df["Pegasus_avg"] = df["text"].apply(lambda x: generate_avg_summary("Pegasus", x))

100%|██████████| 10/10 [01:36<00:00,  9.65s/it]


In [42]:
# get 10 examples
articles_np_10 = articles_np[:10]
articles_np_10.shape

(10,)

In [43]:
# Define a partial new function with the `words` argument fixed to 120
summarize_text_120 = partial(summarize_text, words=250)

# Parallelize the TextRank summarization
with Pool() as pool:
    summarized_articles = pool.map(summarize_text_120, articles_np_10)

In [44]:
df['TextRank (Baseline)_avg'] = summarized_articles

## Evaluating Summarization Models using ROUGE Metrics

In this section, we demonstrate how to evaluate the summarization models using the ROUGE metrics. ROUGE, or Recall-Oriented Understudy for Gisting Evaluation, is a set of metrics and a software package used for evaluating automatic summarization and machine translation software in natural language processing. The metrics compare an automatically produced summary or translation against a reference or a set of references (human-produced) summary or translation. Note that ROUGE is case insensitive, meaning that upper case letters are treated the same way as lower case letters.

The evaluation is performed using the Hugging Face's implementation of ROUGE, which is a wrapper around the Google Research reimplementation of ROUGE.

The `compute_rouge` function, which calculates the ROUGE scores, is located in the `utils.py` file. This file is assumed to have been imported from a GitHub repository, and you can refer to it for more insights on the implementation of the compute_rouge function.

__Defining Model Names__

First, we define a list of model names that we want to evaluate. In this example, we include the TextRank (baseline), BART, T5, and Pegasus models:
`model_names = ["TextRank (Baseline)", "BART", "T5", "Pegasus"]
`

__Computing ROUGE Metrics__

Next, we use the compute_rouge function to calculate the ROUGE scores for each of the models:
`results = compute_rouge(df, model_names)
`

The compute_rouge function takes the DataFrame df and the list of model names as inputs, and returns a dictionary containing the ROUGE scores for each model.

__Creating a DataFrame for Average Metrics__

To better visualize and analyze the results, we can convert the results dictionary to a Pandas DataFrame:
`avg_metrics = pd.DataFrame.from_dict(results)
`

__Displaying the Results__

Finally, we display the DataFrame containing the average ROUGE metrics for each model:
`avg_metrics.T
`

By examining the DataFrame, you can compare the performance of the different summarization models and decide which one best suits your needs.

In [45]:
model_names = ["TextRank (Baseline)", "BART", "T5", "Pegasus"]
results = compute_rouge(df, model_names)

In [46]:
results_bleu = compute_bleu(df, model_names)

In [47]:
avg_metrics = pd.DataFrame.from_dict(results)
avg_metrics = avg_metrics.T

In [48]:
avg_metrics

,rouge1,rouge2,rougeL,rougeLsum
TextRank (Baseline),0.450525,0.200710,0.246021,0.370348
BART,0.449934,0.206406,0.232390,0.311584
T5,0.475323,0.212825,0.238760,0.321936
Pegasus,0.440725,0.192450,0.222895,0.307402


In [49]:
df_bleu = pd.DataFrame.from_dict(results_bleu)
df_bleu = df_bleu.T
df_bleu

,google_bleu
TextRank (Baseline),0.184799
BART,0.166654
T5,0.170961
Pegasus,0.154844


In [50]:
pd.concat([avg_metrics, df_bleu], axis=1)

,rouge1,rouge2,rougeL,rougeLsum,google_bleu
TextRank (Baseline),0.450525,0.200710,0.246021,0.370348,0.184799
BART,0.449934,0.206406,0.232390,0.311584,0.166654
T5,0.475323,0.212825,0.238760,0.321936,0.170961
Pegasus,0.440725,0.192450,0.222895,0.307402,0.154844
